# 自动求导
## 1.简单的例子

In [1]:
#创建变量x并分配初始值

import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [2]:
#现在需要一个地方存储梯度，一个标量函数关于向量x的梯度是向量，并于x的形状相同

x.requires_grad_(True)     #等价于x=torch.arange(4.0, required_grad=True)
x.grad     #默认值是None

In [3]:
#现在计算y

y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [4]:
#x是一个长度为4的向量，计算x和x的点积，得到了y的标量输出。
#通过反向传播函数自动计算y关于x每个分量的梯度，并打印

y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [6]:
#验证梯度计算是否正确

x.grad == 4 * x


tensor([True, True, True, True])

In [7]:
#计算x的另一个函数
#在默认情况下，PyTorch会累积梯度，需要清楚之前的值

x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.], grad_fn=<ZeroBackward0>)

## 2.非标量变量的反向传播

In [8]:
#当y不是标量时，y关于x的导数就是一个矩阵，对于高阶的y和x，求导结果是一个高阶张量
#对于非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度
#在本次例子中，只需求偏导数的和，所以传递一个1的梯度较为合适

x.grad.zero_()
y = x * x
#等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.], grad_fn=<ZeroBackward0>)

## 3.分离计算

In [9]:
#含多元变量的计算

x.grad.zero_()
y = x * x
u = y.detach()      #让u=y的常数形式
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [10]:
#由于记录了y的计算结果，可以随后在y上调用反向传播，得到y=x*x关于x的导数

x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

## 4.Python控制流的梯度计算

In [11]:
#举例

def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [13]:
#计算梯度

a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

In [14]:
a.grad == d / a

tensor(True)